# Final year L6 Project

The following project creates a Convolutional Neural Network based on the following dataset: https://www.kaggle.com/datasets/sartajbhuvaji/brain-tumor-classification-mri

**Importing libraries**

In [1]:
import os
import tensorflow as tf
import cv2 #checkout open cv
import imghdr #allows to check file extensions for images
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import random
import shutil
#import scipy
from sklearn.model_selection import train_test_split
#import kaggle - will likely change

from tensorflow import keras
from tensorflow.python.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten, Dropout
from tensorflow.python.keras.models import sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
#from keras.applications import EfficientNetB0
from keras.layers.pooling.global_average_pooling2d import GlobalAveragePooling2D
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle

In [2]:
testing_path = '/kaggle/input/brain-tumor-classification-mri/Testing'
training_path = '/kaggle/input/brain-tumor-classification-mri/Training'

In [3]:
labels = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']

In [4]:
#Initialising empty lists
x_train = [] 
y_train = [] 

x_test = []
y_test = []

size = 224 #General defined size of the images

for i in labels:  
  # training set
  path = os.path.join(training_path, i)
  for file_name in os.listdir(path):
    img = cv2.imread(os.path.join(path, file_name))
    img = cv2.resize(img, (size, size)) #Resizing images
    img = img.astype('float32') / 255.0 #Normalising values
    x_train.append(img)
    y_train.append(i)

  # testing set
  path = os.path.join(testing_path, i)
  for file_name in os.listdir(path):
    img = cv2.imread(os.path.join(path, file_name))
    img = cv2.resize(img, (size, size))
    img = img.astype('float32') / 255.0
    x_test.append(img)
    y_test.append(i)

#Creating numpy arrays to store labels and images in
x_train = np.array(x_train)
y_train = np.array(y_train)

x_test = np.array(x_test)
y_test = np.array(y_test)

In [5]:
x_train, y_train = shuffle(x_train, y_train, random_state=42)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x_train,y_train, test_size=0.1,random_state=42) #10% validation split

In [7]:
#Creating empty labels to store one-hot values
y_train_onehot = []
y_test_onehot = []

#Appending new labels
for i in y_train:
  y_train_onehot.append(labels.index(i))

#Overwriting labels in previous array
y_train = y_train_onehot

#Using to_categorical to one-hot encode labels
y_train = to_categorical(y_train)


for i in y_test:
  y_test_onehot.append(labels.index(i))

y_test = y_test_onehot

y_test = to_categorical(y_test)

In [8]:
print(y_test)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 ...
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]]


# Initialising callbacks

In [ ]:
%load_ext tensorboard

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, min_delta=0.000, mode='auto', verbose=1)

In [ ]:
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1)

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint("chckpnt.h5",monitor="val_accuracy",save_best_only=True,mode="auto",verbose=1)

In [ ]:
tf_callback = tf.keras.callbacks.TensorBoard(log_dir='logs', histogram_freq=1)

# Building and training the model

In [ ]:
model = keras.Sequential()

model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(units=64, activation='relu'))
model.add(keras.layers.Dropout(0.2))

model.add(keras.layers.Dense(units=4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.1, callbacks=[reduce_lr, tf_callback, checkpoint])

# Evaluation

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)

print('Test accuracy:', test_acc)

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = np.argmax(model.predict(x_test), axis=1)
cm = confusion_matrix(np.argmax(y_test, axis=1), y_pred)
print(cm)
plt.imshow(cm, cmap='Blues')
plt.title('Confusion Matrix')
plt.colorbar()
plt.xticks([0, 1, 2, 3], ['glioma', 'meningioma', 'no_tumor', 'pituitary'])
plt.yticks([0, 1, 2, 3], ['glioma', 'meningioma', 'no_tumor', 'pituitary'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
plt.plot(history.history['val_accuracy'])
plt.plot(history.history['accuracy'])
plt.title('Model Loss and Accuracy')
plt.ylabel('Loss/Accuracy')
plt.xlabel('Epoch')
plt.legend(['val_accuracy', 'accuracy'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['val_loss'])
plt.plot(history.history['loss'])
plt.title('Model Loss and Accuracy')
plt.ylabel('Loss/Accuracy')
plt.xlabel('Epoch')
plt.legend(['val_loss', 'loss'], loc='upper right')
plt.show()

# Exporting the model

In [ ]:
#EXPERIMENTAL
model.save('/kaggle/working/bt_ai_EXP3-91acc-04lss-08-04-23.h5')